In [1]:
import os
from pydub import AudioSegment
import numpy as np

from keras.preprocessing.text import Tokenizer, text_to_word_sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential, Model
from keras.layers import Dense, Embedding, LSTM, Bidirectional, Input, concatenate, Dropout, SpatialDropout1D, Flatten, BatchNormalization, Conv1D, MaxPooling1D
from keras.utils.np_utils import to_categorical
from keras import regularizers
from keras.optimizers import Adam
from keras.callbacks import TensorBoard

C:\Users\Ranet\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


## Raw data preprocessing:

In [2]:
MAP_DIR_PATH = 'MAPS/PROTOTYPE MAPS/'
MAP_NAME = 'Levels'

In [3]:
# Load the beatmap hitcirlces

with open(os.path.join(MAP_DIR_PATH, MAP_NAME + '.osu')) as f:
    content = [x.strip() for x in f.readlines()]

hitobjects_index = [x for x in content].index('[HitObjects]')
X_one_example = '\n'.join([','.join(x.split(',')[:3]) for x in content[hitobjects_index + 1:]])

print(X_one_example[0:200])

376,307,14036
176,192,14888
336,192,15101
200,248,15314
312,136,15527
256,272,15741
256,112,15954
312,248,16167
200,136,16380
56,56,16806
456,328,17445
104,192,18297
412,192,18723
256,28,19362
256,161


In [4]:
length = 13
sequences = list()
for i in range(length, len(X_one_example)):
    # select sequence of tokens
    seq = X_one_example[i-length:i+1]
    # store
    sequences.append(seq)
print('Total Sequences: %d' % len(sequences))

for i in range(5):
    print(sequences[i])
    print('------------')

Total Sequences: 3176
376,307,14036

------------
76,307,14036
1
------------
6,307,14036
17
------------
,307,14036
176
------------
307,14036
176,
------------


In [5]:
chars = sorted(list(set(X_one_example)))
mapping = dict((c, i) for i, c in enumerate(chars))

sequences_mapped = list()
for seq in sequences:
    # integer encode line
    encoded_seq = [mapping[char] for char in seq]
    # store
    sequences_mapped.append(encoded_seq)

for i in range(5):
    print(sequences_mapped[i])
    print('------------')
    


# vocabulary size
vocab_size = len(mapping)
print('Vocabulary Size: %d' % vocab_size)

[5, 9, 8, 1, 5, 2, 9, 1, 3, 6, 2, 5, 8, 0]
------------
[9, 8, 1, 5, 2, 9, 1, 3, 6, 2, 5, 8, 0, 3]
------------
[8, 1, 5, 2, 9, 1, 3, 6, 2, 5, 8, 0, 3, 9]
------------
[1, 5, 2, 9, 1, 3, 6, 2, 5, 8, 0, 3, 9, 8]
------------
[5, 2, 9, 1, 3, 6, 2, 5, 8, 0, 3, 9, 8, 1]
------------
Vocabulary Size: 12


In [6]:
sequences = np.array(sequences_mapped)
X, y = sequences[:,:-1], sequences[:,-1]

print(X[2])
print(y[2])

sequences = [to_categorical(x, num_classes=vocab_size) for x in X]
X = np.array(sequences)
y = to_categorical(y, num_classes=vocab_size)

print(X[2])
print(y[2])

[8 1 5 2 9 1 3 6 2 5 8 0 3]
9
[[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]


In [7]:
# define model
model = Sequential()
model.add(LSTM(75, input_shape=(X.shape[1], X.shape[2])))
model.add(Dense(vocab_size, activation='softmax'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 75)                26400     
_________________________________________________________________
dense_1 (Dense)              (None, 12)                912       
Total params: 27,312
Trainable params: 27,312
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
# compile model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# fit model
model.fit(X, y, epochs=100, verbose=1)

Epoch 1/100
3176/3176 [==============================] - 4s 1ms/step - loss: 2.4341 - acc: 0.1436
Epoch 2/100
3176/3176 [==============================] - 2s 617us/step - loss: 2.2784 - acc: 0.2305
Epoch 3/100
3176/3176 [==============================] - 2s 640us/step - loss: 2.0897 - acc: 0.2824
Epoch 4/100
3176/3176 [==============================] - 2s 625us/step - loss: 1.9130 - acc: 0.3098
Epoch 5/100
3176/3176 [==============================] - 2s 620us/step - loss: 1.8283 - acc: 0.3375
Epoch 6/100
3176/3176 [==============================] - 2s 632us/step - loss: 1.7771 - acc: 0.3495
Epoch 7/100
3176/3176 [==============================] - 2s 632us/step - loss: 1.7642 - acc: 0.3558
Epoch 8/100
3176/3176 [==============================] - 2s 633us/step - loss: 1.7384 - acc: 0.3630
Epoch 9/100
3176/3176 [==============================] - 2s 632us/step - loss: 1.7199 - acc: 0.3700
Epoch 10/100
3176/3176 [==============================] - 2s 627us/step - loss: 1.7002 - acc: 0.3800
E

3176/3176 [==============================] - 2s 619us/step - loss: 0.6307 - acc: 0.8117
Epoch 83/100
3176/3176 [==============================] - 2s 620us/step - loss: 0.6006 - acc: 0.8268
Epoch 84/100
3176/3176 [==============================] - 2s 635us/step - loss: 0.5810 - acc: 0.8372
Epoch 85/100
3176/3176 [==============================] - 2s 631us/step - loss: 0.5670 - acc: 0.8470
Epoch 86/100
3176/3176 [==============================] - 2s 634us/step - loss: 0.5429 - acc: 0.8514
Epoch 87/100
3176/3176 [==============================] - 2s 618us/step - loss: 0.5155 - acc: 0.8630
Epoch 88/100
3176/3176 [==============================] - 2s 623us/step - loss: 0.5048 - acc: 0.8665
Epoch 89/100
3176/3176 [==============================] - 2s 637us/step - loss: 0.4809 - acc: 0.8750
Epoch 90/100
3176/3176 [==============================] - 2s 639us/step - loss: 0.4851 - acc: 0.8759
Epoch 91/100
3176/3176 [==============================] - 2s 619us/step - loss: 0.4470 - acc: 0.8923
Epo

In [11]:
def generate_seq(model, mapping, seq_length, seed_text, n_chars):
    in_text = seed_text
    # generate a fixed number of characters
    for _ in range(n_chars):
        # encode the characters as integers
        encoded = [mapping[char] for char in in_text]
        # truncate sequences to a fixed length
        encoded = pad_sequences([encoded], maxlen=seq_length, truncating='pre')
        # one hot encode
        encoded = to_categorical(encoded, num_classes=len(mapping))
        # predict character
        yhat = model.predict_classes(encoded, verbose=0)
        # reverse map integer to character
        out_char = ''
        for char, index in mapping.items():
            if index == yhat:
                out_char = char
                break
        # append to input
        in_text += char
    return in_text

# test start of rhyme
print(generate_seq(model, mapping, length, '355,355,12356', 1000))

355,355,12356,200

174,192,41600
256,300,64443
256,280,64756
220,286,66709
488,64,670,551924164
2333,38,84770

256,28,47405
256,320,74433
158,288,47598
256,280,72551
202,140,52950
256,44,19004
256,192,19888
336,171,15101
340,234,15667
210,232,88077
473,391,87830
409,118,84007
339,318,80085
439,919,94775
444,144,831,1
220,2362,2593
110,224,37551
332,240,22555
160,240,52299
312,122,43268
156,128,40906
332,286,70859
128,292,22788
256,192,27840
356,492,74541
272,88,26666
256,192,88473
336,192,47811
320,336,80864
279,219,87670
330,192,66403
256,300,64453
256,280,64556
220,180,92775
58,444,3732
256,1404,3251,244,12
2

40,286,99911
422,43,39518
224,,76,703
256,256,5640

429,192,30155
392,104,22741
364,288,26588
336,291,84107
433,111,40157
144,
36,02301
448,248,3651,
252
2604,112,19984
6,14172622,3,32,400
15248,408,2271

280,288,7989

48,140,87071
39,2111,85298
332,305,40447
176,292,40993
132,88,40335
256,280,42459
328,288,8299

256,57,30464
256,192,34751
172,336,85887
336,215,79166
256,200,48